
<img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to Fetch Jenkins Build Logs using unSkript legos.</b>
</div>

<br>

<center><h2>Fetch Jenkins Build Logs</h2></center>

# Steps Overview
    1) Get logs for the particular job name.
    2) Post the logs to Slack channel.

Here we will use unSkript Get Jenkins Logs from a job Lego. This lego takes job_name and build_number as input. This input is used to discover logs from Jenkins using a Job.

In [ ]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional


from beartype import beartype
@beartype
def jenkins_get_logs(handle, job_name: str, build_number: int = 0):
    """jenkins_get_logs returns logs for the particular job name.
        :type nbParamsObj: object
        :param nbParamsObj: Object containing global params for the notebook.
        :type credentialsDict: dict
        :param credentialsDict: Dictionary of credentials info.
        :rtype: Dict with builds number and logs.
    """

    # Input param validation.
    job = handle.get_job(job_name)

    if build_number == 0:
        res = job.get_last_build()
        return res.get_console()

    res = job.get_build(build_number)
    return res.get_console()


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "build_number": "build_number",
    "job_name": "job_name"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(jenkins_get_logs, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

Here we will use unSkript Post Slack Message Lego. This lego takes channel: str and message: str as input. This input is used to post the message to the slack channel.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)


from beartype import beartype
def legoPrinter(func):
    def Printer(*args, **kwargs):
        output = func(*args, **kwargs)
        if output:
            channel = kwargs["channel"]
            pp.pprint(print(f"Message sent to Slack channel {channel}"))
        return output
    return Printer


@legoPrinter
@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> bool:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return True
    except SlackApiError as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        return False
    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return False


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "channel": "channel",
    "message": "f\\"Jenkins logs for Job:{job_name} Build:{build_number} {jenkins_output}\\""
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(slack_post_message, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

### Conclusion
In this Runbook, we demonstrated the use of unSkript's Jenkins and slack legos to perform Jenkins action and this runbook fetches the logs for a given Jenkins job and posts to a slack channel. To view the full platform capabilities of unSkript please visit https://unskript.com